In [1]:
import pandas as pd
import numpy as np
from hyperopt import fmin, tpe, hp
import ipynb.fs.full.utils as utils
from ipynb.fs.full.features import features_independientes_precio, features_dependientes_precio, features_de_csvs

df_train = pd.read_csv('./data/train.csv')
# Para usarse con el submit a Kaggle
df_eval = pd.read_csv('./data/test.csv')

df_train, df_eval = features_de_csvs(df_train, df_eval)

# Randoms solo para asegurarse que los features esten bien hechos
# df_train = df_train.sample(frac=1).reset_index(drop=True)
# df_train = utils.dolarizar_df(df_train)
# df_train = df_train.sample(frac=1).reset_index(drop=True)

df_train, df_test = utils.dividir_df_testeo(df_train, test_size=0.15)

# df_train = df_train.sample(frac=1).reset_index(drop=True)
# df_test = utils.pesificar_df(df_test)
# df_train = df_train.sample(frac=1).reset_index(drop=True)


In [2]:
def llenar_nulls(df):
    df['metrostotales'] = df['metrostotales'].fillna(df['metroscubiertos'])
    df['metroscubiertos'] = df['metrostotales'].fillna(df['metrostotales'])
    
    df['habitaciones'] = df['habitaciones'].fillna(df['habitaciones'].mean())
    df['garages'] = df['garages'].fillna(df['garages'].mean())
    df['banos'] = df['banos'].fillna(df['banos'].mean())
    
    df['fecha'] = pd.to_datetime(df['fecha']).astype(int)

df_test_f = features_independientes_precio(df_test)
df_test_f = features_dependientes_precio(df_test_f, df_train)

df_train_f = features_independientes_precio(df_train)
df_train_f = features_dependientes_precio(df_train_f, df_train)

llenar_nulls(df_train_f)
llenar_nulls(df_test_f)

In [10]:
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
    
def normalizar_df(df, features):
    min_max = MinMaxScaler()
    df[features] = pd.DataFrame(min_max.fit_transform(df[features]), columns=features)
    return df
    


features = ['habitaciones', 'garages', 'banos', 'fecha',
       'metroscubiertos', 'metrostotales', 'provincia',
       'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas', 'centroscomercialescercanos']

features_test = ['prop_frecuente', 'top_provincia', 'porcentaje_metros', 'diferencia_metros', 'promedio_precio_ciudad', 
                 'promedio_por_mes', 'anio', 'promedio_id_zona', 'promedio_precio_tipo_propiedad', 
                 'promedio_precio_hbg_tipo_propiedad', 'count_id_zona', 'count_ciudad', 'puntaje', 
                 'count_tipo_propiedad', 'count_tipo_propiedad_ciudad', 'promedio_id_zona_gen', 
                 'promedio_precio_tipo_propiedad_ciudad_gen', 'promedio_precio_hbg_tipo_propiedad_provincia']

features_cat = ['provincia', 'tipodepropiedad']

features += features_test + features_cat

df_train_g = utils.filtrar_features(df_train_f, features, 'precio')

features_a_normalizar = [f for f in features if f not in features_cat + ['precio']]

df_train_n = pd.get_dummies(df_train_g, columns=features_cat)
df_train_n = normalizar_df(df_train_n, features_a_normalizar)

x_train, x_test, y_train, y_test = utils.dividir_dataset(df_train_n, 'precio', test_size=0.15)

model = Sequential()
# model.add(Dense(units=1024, activation='relu'))
# model.add(Dense(units=256, activation='relu'))
model.add(Dense(units=128, activation='relu'))
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=32, activation='relu'))
model.add(Dense(units=1, activation='linear'))

model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['accuracy'])


model.fit(x_train.values, y_train.values, epochs=25, batch_size=32)

y_pred_test = model.predict(x_test, batch_size=32 )

keras_mae_test = utils.MAE(y_test, y_pred_test)
print(f"MAE Keras (test): {keras_mae_test:.5f}")

Epoch 1/25
173400/173400 [==============================] - 8s 44us/step - loss: 1126110.2104 - accuracy: 0.0000e+00
Epoch 2/25
173400/173400 [==============================] - 8s 44us/step - loss: 705705.5780 - accuracy: 0.0000e+00
Epoch 3/25
173400/173400 [==============================] - 8s 44us/step - loss: 657598.7934 - accuracy: 0.0000e+00
Epoch 4/25
173400/173400 [==============================] - 8s 44us/step - loss: 639650.4090 - accuracy: 0.0000e+00
Epoch 5/25
173400/173400 [==============================] - 8s 44us/step - loss: 628442.9315 - accuracy: 0.0000e+00
Epoch 6/25
173400/173400 [==============================] - 8s 44us/step - loss: 620444.2586 - accuracy: 0.0000e+00
Epoch 7/25
173400/173400 [==============================] - 8s 44us/step - loss: 613978.6702 - accuracy: 0.0000e+00
Epoch 8/25
173400/173400 [==============================] - 8s 45us/step - loss: 608776.0410 - accuracy: 0.0000e+00
Epoch 9/25
173400/173400 [==============================] - 8s 44us/ste